### EJERCICIO 1

La empresa Opta (https://www.optasports.com/) proporciona información detallada
acerca de los eventos que ocurren en partidos de fútbol, tales como pases, tiros,
tarjetas, goles, penaltis, etc.
Los datos se proporcionan en formato XML, con la siguiente estructura:
|-- Games
| |-- Game
| | |-- Event
| | |-- Q

Cada evento Event contiene un atributo type_id que identifica el tipo de evento
(En este ejercicio, nos centraremos en los pases, que tienen type_id="1"). El resto
de atributos de los campos Event de interés se detallan a continuación:
• period_id - Primera (1) o segunda (2) parte del partido
• min - Minuto del partido en el que se produjo el evento
• sec - Segundo del partido en el que se produjo el evento
• outcome - En los pases, outcome="1" indica que el pase fue exitoso
• x - Valor de la coordenada x en el campo donde se produjo el evento
• y - Valor de la coordenada y en el campo donde se produjo el evento

Dentro de cada evento, existen una serie de cualificadores Q que proporcionan
información detallada de cada uno de los eventos. Para este ejercicio, nos centraremos
en el atributo qualifier_id, concretamente en dos valores:
* qualifier_id = "140" – Coordenada x final del pase
* qualifier_id = "141" – Coordenada y final del pase

A partir del fichero OptaF24.xml almacenado en un servidor FTP, que contiene los eventos
de un partido concreto, crear un dataframe que contenga todos los pases de dicho partido.
Este dataframe deberá contener los siguientes campos (columnas):
    
* Columna Descripción
* team Nombre del equipo que realizó el pase
* half Primer o segundo tiempo
* minute Minuto del partido
* second Segundo del partido
* x_origin Coordenada x del inicio del pase
* y_origin Coordenada y del inicio del pase
* x_destination Coordenada x del final del pase
* y_destination Coordenada y del final del pase
* outcome Indica si el pase tuvo éxito (1) o no (0)

1) Descarga el archivo OptaF24.xml creando una conexión al servidor FTP
* Servidor: f25-preview.runhosting.com
* Usuario: 3185129_EOI
* Contraseña: FTP_EOI1920



In [29]:
import pandas as pd
import xml.etree.ElementTree as ET
from ftplib import FTP


host =  'f25-preview.runhosting.com'
user = '3185129_EOI'
passwd= 'FTP_EOI1920'
ftp = FTP(host, user, passwd)

ftp.dir()
filename= "OptaF24.xml"
ftp.retrbinary("RETR " +filename,
              open(filename,"wb").write)


drwxr-xr-x   2 3185129_EOI 1111           58 Nov 24  2019 .
drwxr-xr-x   2 3185129_EOI 1111           58 Nov 24  2019 ..
-rw-r--r--   1 3185129_EOI 1111        33182 Nov 24  2019 books.xml
-rw-r--r--   1 3185129_EOI 1111         3841 Oct 20  2019 colors.json
-rw-r--r--   1 3185129_EOI 1111       763509 Oct 20  2019 OptaF24.xml


'226 Transfer complete'

2) Crea una lista inicialmente vacía para cada de las columnas del dataframe


In [30]:
tree = ET.parse(filename)
raiz = tree.getroot()
   
team = []
half = []
minute = []
second = []    
x_origin = []
y_origin = []
x_destination = []
y_destination = []
outcome = []

3) Itera cada evento y filtra aquellos que correspondan a pases (atributo type_id="1"), añadiendo a cada lista del paso anterior los atributos correspondientes de cada evento.


In [34]:
equipo = {raiz[0].attrib["home_team_id"]: raiz[0].attrib["home_team_name"],
            raiz[0].attrib["away_team_id"]: raiz[0].attrib["away_team_name"]}


for hijo in raiz:    
    for evento in hijo:
        if evento.attrib.get("type_id") == '1':
            team.append(equipo[evento.attrib.get("team_id")])  
            half.append(evento.attrib.get("period_id"))
            minute.append(evento.attrib.get("min"))
            second.append(evento.attrib.get("sec"))
            x_origin.append(float(evento.attrib.get("x")))
            y_origin.append(evento.attrib.get("y"))
            outcome.append(evento.attrib.get("outcome"))            
            for qualifier in evento:               
                if qualifier.attrib.get("qualifier_id") == "140":
                    x_destination.append(float(qualifier.attrib.get("value")))
                if qualifier.attrib.get("qualifier_id") == "141":
                    y_destination.append(qualifier.attrib.get("value"))
                
column_titles = ["team", "half", "min", "second","x_origin", "y_origin", "x_destination", "y_destination", "outcome"]
table = pd.DataFrame(data=[team, half, minute,second, x_origin, y_origin, x_destination, y_destination, outcome], index=column_titles)            
table = table.T
print(table)

                  team half min second x_origin y_origin x_destination  \
0      Manchester City    1   0      1     50.1     50.0          52.4   
1      Manchester City    1   0      2     48.2     49.1            29   
2      Manchester City    1   0     19     27.8    100.0          49.3   
3     Bolton Wanderers    1   0     23     50.9     20.0          63.6   
4      Manchester City    1   0     30     29.3     73.0          30.2   
...                ...  ...  ..    ...      ...      ...           ...   
1887  Bolton Wanderers    2  93     42     16.3     82.5          71.7   
1888   Manchester City    2  93     45     28.6     27.6          34.2   
1889   Manchester City    2  93     46     34.4      9.1            63   
1890   Manchester City    2  93     47     62.4     20.7          61.8   
1891  Bolton Wanderers    2  94      4        5     63.0          68.1   

     y_destination outcome  
0             49.1       1  
1             76.5       1  
2             93.4      

4) Obtén los pases de más de 20 metros de avance en la coordenada x. ¿Qué equipo ha efectuado más veces estos pases?   


In [33]:
table['long_pase']=table['x_destination'].subtract(table['x_origin'])
table_2 = table[table['long_pase']>20.0]
table_2[['team','long_pase']].groupby('team').count()

,long_pase
team,
Bolton Wanderers,89
Manchester City,74
